In [41]:
import numpy as np
import itertools

In [42]:
i1 = 0.05
i2 = 0.10

b1 = 0.35

w1 = 0.15
w2 = 0.20
w3 = 0.25
w4 = 0.30

o1 = 0.01
o2 = 0.99

b2 = 0.60

w5 = 0.4
w6 = 0.45
w7 = 0.5
w8 = 0.55

In [43]:
def sigmoid(z):
    
    return 1 / (1 + np.exp(-z))

In [44]:
def calculate_weight_set_dimensions(dimensions):

    a, b = itertools.tee(dimensions[::-1])
    next(b, None)
    weight_set_dimensions = list(zip(a, b))[::-1]
    
    return weight_set_dimensions

In [45]:
dimensions = [2,2,2]

calculate_weight_set_dimensions(dimensions)

[(2, 2), (2, 2)]

In [46]:
def initialise_weights(dimensions):
    
    # For single hidden layer neural network there will be 2 sets of weights;
    # 1- one set to hidden layer
    # 2- one set from hidden layer
    # number of weight sets = no_of_hidden_layers + 1
    
    weight_dims = calculate_weight_set_dimensions(dimensions)
    no_of_weight_sets = len(weight_dims)
    
    # W_set holds weight sets such as w1, w2, w3 etc.
    W = np.empty_like(range(no_of_weight_sets), dtype=object)
    B = np.empty_like(range(no_of_weight_sets), dtype=object)
    for index, (row, column) in enumerate(weight_dims):
        W[index] = np.random.rand(row, column)
        B[index] = np.random.rand()
    return W, B

In [47]:
def forwardpass(input_vals, W, B):
    
    weight_dims = calculate_weight_set_dimensions(dimensions)
    no_of_weight_sets = len(weight_dims)
    
    Z = np.empty_like(range(no_of_weight_sets + 1), dtype=object)
    A = np.empty_like(range(no_of_weight_sets + 1), dtype=object)
    A[0] = input_vals
    Z[0] = None
    B[0] = 0.35
    B[1] = 0.6
    for index in range(no_of_weight_sets):
        Z[index + 1] = W[index] @ A[index] + B[index]
        A[index + 1] = sigmoid(Z[index + 1])
    return A, Z

In [48]:
def calculate_error(y, y_hat):
    return 0.5 * np.square(y - y_hat)

In [50]:
input_vals = np.array([0.05, 0.10]).reshape((dimensions[0], 1))
output_vals = np.array([0.01, 0.99]).reshape((dimensions[-1], 1))

W, B = initialise_weights(dimensions)

initial_weights = W[0]
initial_weights[0][0] = 0.15
initial_weights[0][1] = 0.2
initial_weights[1][0] = 0.25
initial_weights[1][1] = 0.30
B[0] = 0.35

second_set_of_weights = W[1]
second_set_of_weights[0][0] = 0.4
second_set_of_weights[0][1] = 0.45
second_set_of_weights[1][0] = 0.5
second_set_of_weights[1][1] = 0.55
B[1] = 0.6

A, Z = forwardpass(input_vals, W, B)
E = calculate_error(output_vals, A[-1])

In [51]:
E

array([[0.27481108],
       [0.02356003]])